In [ ]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from transformers import BertTokenizer, BertModel
from torch.nn import Linear, CrossEntropyLoss
from torch.optim import Adam
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score,precision_score,recall_score
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/Финальный_датафрейм.csv')
df = df.replace({'score':{7:0,8:1, 9:1, 10:1}})

Mounted at /content/drive


In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

X_train = train_df['prepared_text'].values
y_train = train_df['score'].values

if X_train.ndim == 1:
    X_train = X_train.reshape(-1, 1)

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [ ]:
class TextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, item):
        text = str(self.texts[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')

train_dataset = TextDataset(
    texts=X_resampled,
    labels=y_resampled,
    tokenizer=tokenizer
)

test_dataset = TextDataset(
    texts=test_df['prepared_text'].to_numpy(),
    labels=test_df['score'].to_numpy(),
    tokenizer=tokenizer
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

In [ ]:
# Data Loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64)

In [ ]:
model = BertForSequenceClassification.from_pretrained('DeepPavlov/rubert-base-cased', num_labels=2)
if torch.cuda.is_available():
    model.cuda()

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

In [ ]:
from tqdm import tqdm

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Функция обучения
def train_epoch(model, data_loader, optimizer):
    model.train()
    total_loss = 0
    for d in data_loader:
        input_ids = d["input_ids"].to(torch.device('cuda'))
        attention_mask = d["attention_mask"].to(torch.device('cuda'))
        labels = d["labels"].to(torch.device('cuda'))

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(data_loader)

# Обучение модели
num_epochs = 5
for epoch in tqdm(range(num_epochs)):
    train_loss = train_epoch(model, train_loader, optimizer)
    print(f'Epoch {epoch + 1}, Train Loss: {train_loss:.4f}')

 20%|██        | 1/5 [21:39<1:26:36, 1299.23s/it]

Epoch 1, Train Loss: 0.6968


 40%|████      | 2/5 [42:53<1:04:13, 1284.49s/it]

Epoch 2, Train Loss: 0.6950


 60%|██████    | 3/5 [1:04:05<42:37, 1278.61s/it]

Epoch 3, Train Loss: 0.6942


 80%|████████  | 4/5 [1:25:18<21:16, 1276.73s/it]

Epoch 4, Train Loss: 0.6940


100%|██████████| 5/5 [1:46:27<00:00, 1277.49s/it]

Epoch 5, Train Loss: 0.6950


In [ ]:
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(f"Trainable: {name}")
#     else:
#         print(f"Frozen: {name}")

In [ ]:
import numpy as np
from scipy.special import softmax

model.eval()
predictions, true_labels = [], []

with torch.no_grad():
    for d in test_loader:
            input_ids = d["input_ids"].cuda()
            attention_mask = d["attention_mask"].cuda()
            labels = d["labels"].cuda()

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits.detach().cpu().numpy()
            label_ids = labels.to('cpu').numpy()

            predictions.extend(softmax(logits, axis=1))
            true_labels.extend(label_ids.flatten())

In [ ]:
torch.save(model, 'full_bert_russian_classification.pt')

In [ ]:
probs = []
for pred in predictions:
  probs.append(pred[1])

probs

[0.49356085,
 0.48106194,
 0.49510217,
 0.4896295,
 0.5257802,
 0.49598262,
 0.4950325,
 0.48756492,
 0.49660203,
 0.5040051,
 0.49558187,
 0.5128205,
 0.49300793,
 0.4774986,
 0.48234475,
 0.49478194,
 0.48065227,
 0.49777016,
 0.49514765,
 0.48895642,
 0.49618384,
 0.49046856,
 0.502496,
 0.5105962,
 0.48507932,
 0.48189676,
 0.48935893,
 0.49505988,
 0.48960257,
 0.5013272,
 0.48697668,
 0.49970216,
 0.49654362,
 0.48717827,
 0.49658248,
 0.488199,
 0.5061926,
 0.4855152,
 0.48959315,
 0.48268616,
 0.48696175,
 0.5054543,
 0.49388307,
 0.5024815,
 0.495499,
 0.49629325,
 0.49083295,
 0.4843903,
 0.49693406,
 0.49276564,
 0.4999334,
 0.49285138,
 0.4974035,
 0.4881342,
 0.47688892,
 0.49476668,
 0.48432633,
 0.48654112,
 0.5005878,
 0.4969558,
 0.48778418,
 0.50450903,
 0.4871397,
 0.48726177,
 0.50303,
 0.5024248,
 0.48601532,
 0.49420375,
 0.4943051,
 0.5097339,
 0.48014355,
 0.5047016,
 0.4826211,
 0.50226563,
 0.49272895,
 0.50454307,
 0.4889342,
 0.49487394,
 0.5042925,
 0.49638

In [ ]:
print(f1_score(true_labels, np.array(probs) < .5))
print(accuracy_score(true_labels, np.array(probs) < .5))
print(precision_score(true_labels, np.array(probs) < .5))
print(recall_score(true_labels, np.array(probs) < .5))

0.7820844099913867
0.6510344827586206
0.7978910369068541
0.7668918918918919
